In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('Twitter_Data.csv')

In [3]:
df.head()

,review,label
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [4]:
df.label.value_counts()

 1.0    72250
 0.0    55213
-1.0    35510
Name: label, dtype: int64

In [5]:
df['label']=df['label'].apply(lambda x: 0 if x==0 else (1 if x==1 else 2)) # rating sütununa göre etiket belirlenir
df['review'] = df['review'].astype('str') 

In [6]:
df.head()

,review,label
0,when modi promised “minimum government maximum...,2
1,talk all the nonsense and continue all the dra...,0
2,what did just say vote for modi welcome bjp t...,1
3,asking his supporters prefix chowkidar their n...,1
4,answer who among these the most powerful world...,1


In [7]:
df.label.value_counts()

1    72250
0    55213
2    35517
Name: label, dtype: int64

In [8]:
from nltk.corpus import stopwords
from textblob import Word

sw=stopwords.words("english")

In [164]:
def data_cleaning(df,choice):
    df = df.applymap(lambda s: s.lower() if type(s) == str else s) # Tüm metinler küçük harf formatına dönüştürülür
    df["review"]=df["review"].str.replace("[^\w\s]","") # noktalama işaretlerinin silinmesi
    df["review"]=df["review"].str.replace("\d","") # sayıların silinmesi
    df["review"]=df["review"].apply(lambda x: " ".join(x for x in x.split() if x not in sw)) # stopwords kelimelerin silinmesi
    if choice == 1:
        df["review"]=df["review"].apply(lambda x: " ".join(Word(i).lemmatize() for i in x.split())) # lemmatization
    elif choice == 2:
        df["review"]=df["review"].apply(lambda x: " ".join(Word(i).stem() for i in x.split())) # stemming
    return df['review']

In [10]:
df['review']=data_cleaning(df, 1)

C:\Users\OUZHAN~1\AppData\Local\Temp/ipykernel_1384/57265534.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["review"]=df["review"].str.replace("[^\w\s]","") # noktalama işaretlerinin silinmesi
C:\Users\OUZHAN~1\AppData\Local\Temp/ipykernel_1384/57265534.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["review"]=df["review"].str.replace("\d","") # sayıların silinmesi


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
def feature_extraction(df,max_features):
    print("TF-IDF yöntemi seçildi")
    vectorizer = TfidfVectorizer(lowercase=False,ngram_range=(1,2),dtype=np.byte,max_features=max_features)
    X = vectorizer.fit_transform(df['review'].tolist())
    return X,vectorizer

In [13]:
X,vectorizer=feature_extraction(df,None)

TF-IDF yöntemi seçildi


C:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:2029: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int8'> 'dtype' will be converted to np.float64.
  warnings.warn(


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,df['label'],random_state=1,test_size=0.25)

In [16]:
from sklearn.metrics import accuracy_score, classification_report

In [17]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
model_lr=lr.fit(X_train,y_train)
prediction_lr=model_lr.predict(X_test)
print("Accuracy Score on traning data: ",lr.score(X_train,y_train))
print("Accuracy Score on test data: ",lr.score(X_test,y_test))
print(classification_report(y_test, prediction_lr))
print("Accuracy Score: ", accuracy_score(y_test, prediction_lr))

C:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy Score on traning data:  0.939436331656236
Accuracy Score on test data:  0.8519327524849675
              precision    recall  f1-score   support

           0       0.81      0.94      0.87     13910
           1       0.88      0.88      0.88     17972
           2       0.87      0.66      0.75      8863

    accuracy                           0.85     40745
   macro avg       0.85      0.83      0.83     40745
weighted avg       0.86      0.85      0.85     40745

Accuracy Score:  0.8519327524849675


In [18]:
from sklearn.linear_model import RidgeClassifier
rc=RidgeClassifier()
model_rc=rc.fit(X_train,y_train)
prediction_rc=model_rc.predict(X_test)
print("Accuracy Score on traning data: ",rc.score(X_train,y_train))
print("Accuracy Score on test data: ",rc.score(X_test,y_test))
print(classification_report(y_test, prediction_rc))
print("Accuracy Score: ", accuracy_score(y_test, prediction_rc))

Accuracy Score on traning data:  0.9938397349368021
Accuracy Score on test data:  0.8520554669284575
              precision    recall  f1-score   support

           0       0.81      0.94      0.87     13910
           1       0.89      0.86      0.88     17972
           2       0.86      0.70      0.77      8863

    accuracy                           0.85     40745
   macro avg       0.85      0.83      0.84     40745
weighted avg       0.86      0.85      0.85     40745

Accuracy Score:  0.8520554669284575


In [19]:
from sklearn.linear_model import Perceptron
pt=Perceptron()
model_pt=pt.fit(X_train,y_train)
prediction_pt=model_pt.predict(X_test)
print("Accuracy Score on traning data: ",pt.score(X_train,y_train))
print("Accuracy Score on test data: ",pt.score(X_test,y_test))
print(classification_report(y_test, prediction_pt))
print("Accuracy Score: ",accuracy_score(y_test, prediction_pt))

Accuracy Score on traning data:  0.9988792080827913
Accuracy Score on test data:  0.8636887961713093
              precision    recall  f1-score   support

           0       0.83      0.94      0.88     13910
           1       0.91      0.87      0.89     17972
           2       0.84      0.74      0.79      8863

    accuracy                           0.86     40745
   macro avg       0.86      0.85      0.85     40745
weighted avg       0.87      0.86      0.86     40745

Accuracy Score:  0.8636887961713093


In [20]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
model_xgb=xgb.fit(X_train,y_train)
prediction_xgb=model_xgb.predict(X_test)
print("Accuracy Score on traning data: ",xgb.score(X_train,y_train))
print("Accuracy Score on test data: ",xgb.score(X_test,y_test))
print(classification_report(y_test, prediction_xgb))
print("Accuracy Score: ",accuracy_score(y_test, prediction_xgb))

Accuracy Score on traning data:  0.8427046263345196
Accuracy Score on test data:  0.8229721438213278
              precision    recall  f1-score   support

           0       0.75      0.96      0.84     13910
           1       0.89      0.82      0.85     17972
           2       0.86      0.62      0.72      8863

    accuracy                           0.82     40745
   macro avg       0.83      0.80      0.80     40745
weighted avg       0.83      0.82      0.82     40745

Accuracy Score:  0.8229721438213278


In [21]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()
model_lgbm=lgbm.fit(X_train,y_train)
prediction_lgbm=model_lgbm.predict(X_test)
print("Accuracy Score on traning data: ",lgbm.score(X_train,y_train))
print("Accuracy Score on test data: ",lgbm.score(X_test,y_test))
print(classification_report(y_test, prediction_lgbm))
print("Accuracy Score: ",accuracy_score(y_test, prediction_lgbm))

Accuracy Score on traning data:  0.8701272139730847
Accuracy Score on test data:  0.8619217081850534
              precision    recall  f1-score   support

           0       0.80      0.97      0.88     13910
           1       0.92      0.86      0.89     17972
           2       0.87      0.71      0.78      8863

    accuracy                           0.86     40745
   macro avg       0.86      0.84      0.85     40745
weighted avg       0.87      0.86      0.86     40745

Accuracy Score:  0.8619217081850534


In [22]:
import tweepy, codecs

In [188]:
def hashtag_df(key, count):
    consumer_key = 'bIQ3GqTGdNIS9qtYATf1f2kw3'
    consumer_secret = 'HuF1e9E6CRT1QmvOml3Os3cUnX3FxSbSzkweMM6nztdGGprU26'
    access_token = '100390695-WeMfH504k2zeXyozN3EAqpM93Qxjts6sfF3BKjkh'
    access_token_secret = 'VfGMaqkkRbtUhNmmuYr55MZTtwgOPtNuyB5OVhO0gbIss'
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    API = tweepy.API(auth)
    tweetler=API.search_tweets(q=key, lang="en", result_type="recent", count=count)
    df2 = pd.DataFrame()
    df2["review"] = [tweet.text for tweet in tweetler]
    return df2

In [262]:
key = input('Input a key:')

Input a key:covid


In [263]:
count = input('Input a count value:')

Input a count value:5


In [264]:
df2 = hashtag_df(key, count)

In [265]:
df2

,review
0,RT @vlamers: Just tuned into #ACIP and the dat...
1,My 1 yr old with Covid AND teething finally sl...
2,RT @CNN: CDC advisers vote to recommend Modern...
3,RT @_InsertPunHere_: I had a milkshake last ni...
4,RT @JQT_web3vc: I don’t just make bear market ...


In [266]:
df3 = df2.copy()

In [267]:
df2.review = data_cleaning(df2,1)

C:\Users\OUZHAN~1\AppData\Local\Temp/ipykernel_1384/389565704.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["review"]=df["review"].str.replace("[^\w\s]","") # noktalama işaretlerinin silinmesi
C:\Users\OUZHAN~1\AppData\Local\Temp/ipykernel_1384/389565704.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df["review"]=df["review"].str.replace("\d","") # sayıların silinmesi


In [268]:
df2.review = df2.review.str.replace("rt","")

In [269]:
df2

,review
0,vlamers tuned acip data emotionally exhaustin...
1,yr old covid teething finally sleep mother htt...
2,cnn cdc adviser vote recommend moderna pfizer...
3,_insepunhere_ milkshake last night went gym f...
4,jqt_webvc dont make bear market call here bul...


In [270]:
test_data = vectorizer.transform(df2.review)

In [271]:
print('Logistic Regression Classifier Prediction:',model_lr.predict(test_data))
print('Ridge Classifier Prediction:',model_rc.predict(test_data))
print('Perceptron Classifier Prediction:',model_pt.predict(test_data))
print('XGBoost Classifier Prediction:',model_xgb.predict(test_data))
print('LightGBM Classifier Prediction:',model_lgbm.predict(test_data))

Logistic Regression Classifier Prediction: [1 1 1 1 1]
Ridge Classifier Prediction: [1 1 1 1 1]
Perceptron Classifier Prediction: [1 1 1 1 1]
XGBoost Classifier Prediction: [1 1 1 1 0]
LightGBM Classifier Prediction: [1 1 1 1 0]


In [273]:
df2.review[1]

'yr old covid teething finally sleep mother httpstcovtjqlcele'